## Environment setup

In [3]:
import glob, os, time
from datetime import datetime
from functools import partial
import importlib

import numpy as np

import tensorflow as tf

## Networks definitions and adaptations

In [1]:
from architecture_definitions import *
 
dir_postfix = get_classic_nn_dir_postfix() 
    
print_Classic_NN()  

Classic NN definitions:
dense_layer1_size: 128
dense_layer2_size: 64
dense_layer3_size: 48
dense_layer4_size: 1
------------------------


### Training data set preparation

In [7]:
import io_functions as io
importlib.reload(io)

batchSize = 4096
nEpochs = 1

trainDataDir = "/scratch_ssd/akalinow/ProgrammingProjects/MachineLearning/OMTF/data/18_12_2020/"   
trainFileNames = glob.glob(trainDataDir+'OMTFHits_pats0x0003_oldSample_files_*_chunk_0.tfrecord.gzip')

trainDataDir = "/scratch_cmsse/alibordi/data/"
trainFileNames = glob.glob(testDataDir+'*22Feb*tfrecord.gzip')

dataset = io.get_Classic_NN_dataset(batchSize, nEpochs, trainFileNames, isTrain=True)

Reading data from files:
/scratch_cmsse/alibordi/data/omtfAnalysis_22Feb.tfrecord.gzip


### Model definition

In [5]:
import model_functions as models
importlib.reload(models)

import io_functions as io
importlib.reload(io)

networkInputSize = 2 * np.sum(io.getFeaturesMask()) + 1
loss_fn = 'mae'

model = models.get_Classic_NN(networkInputSize=networkInputSize, loss_fn=loss_fn)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pt_layer_1 (Dense)          (None, 128)               4864      
                                                                 
 pt_layer_2 (Dense)          (None, 64)                8256      
                                                                 
 pt_layer_3 (Dense)          (None, 48)                3120      
                                                                 
 dense (Dense)               (None, 1)                 49        
                                                                 
Total params: 16,289
Trainable params: 16,289
Non-trainable params: 0
_________________________________________________________________


### The training loop

In [8]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2070 SUPER, compute capability 7.5
Compute dtype: float16
Variable dtype: float32


2023-03-16 17:03:41.384307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [8]:
%%time
current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training start. Current Time =", current_time)

nEpochs = 50

log_dir = "logs/fit/" + current_time + dir_postfix
job_dir = "training/" + current_time + dir_postfix

checkpoint_path = job_dir + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq = 5085)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=(10, 20))

model.save_weights(checkpoint_path.format(epoch=0))
   
model.fit(dataset, epochs=nEpochs, shuffle=True,
            callbacks=[tensorboard_callback, cp_callback]
            )
model.save(job_dir, save_format='tf')

current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training end. Current Time =", current_time)

Training start. Current Time = 2023_Mar_17_17_13_49
Epoch 1/50


2023-03-17 17:13:49.423655: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2023-03-17 17:13:49.423678: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2023-03-17 17:13:49.529906: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2023-03-17 17:13:49.531184: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed


     10/Unknown - 1s 17ms/step - loss: 314.3122

2023-03-17 17:13:49.995231: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2023-03-17 17:13:49.995260: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2023-03-17 17:13:50.197016: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-03-17 17:13:50.200834: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2023-03-17 17:13:50.357353: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 1495 callback api events and 1474 activity events. 
2023-03-17 17:13:50.423990: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2023-03-17 17:13:50.608506: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/2023_Mar_17_17_13_49_classic_128_64_48_1/plugins/profile/2023_03_17_17_13_50



     29/Unknown - 1s 36ms/step - loss: 120.1534

2023-03-17 17:13:50.820561: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/2023_Mar_17_17_13_49_classic_128_64_48_1/plugins/profile/2023_03_17_17_13_50/282230e604e0.trace.json.gz
2023-03-17 17:13:50.918841: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/2023_Mar_17_17_13_49_classic_128_64_48_1/plugins/profile/2023_03_17_17_13_50

2023-03-17 17:13:50.923955: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/fit/2023_Mar_17_17_13_49_classic_128_64_48_1/plugins/profile/2023_03_17_17_13_50/282230e604e0.memory_profile.json.gz
2023-03-17 17:13:50.926164: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/fit/2023_Mar_17_17_13_49_classic_128_64_48_1/plugins/profile/2023_03_17_17_13_50
Dumped tool data for xplane.pb to logs/fit/2023_Mar_17_17_13_49_classic_128_64_48_1/plugins/profile/

96/96 [==============================] - 3s 26ms/step - loss: 49.6816
Epoch 2/50
96/96 [==============================] - 2s 22ms/step - loss: 16.5711
Epoch 3/50
96/96 [==============================] - 2s 21ms/step - loss: 13.5054
Epoch 4/50
96/96 [==============================] - 2s 21ms/step - loss: 11.8692
Epoch 5/50
96/96 [==============================] - 2s 22ms/step - loss: 10.9808
Epoch 6/50
96/96 [==============================] - 2s 22ms/step - loss: 10.4205
Epoch 7/50
96/96 [==============================] - 2s 22ms/step - loss: 9.9278
Epoch 8/50
96/96 [==============================] - 2s 21ms/step - loss: 9.6024
Epoch 9/50
96/96 [==============================] - 2s 22ms/step - loss: 9.3255
Epoch 10/50
96/96 [==============================] - 2s 21ms/step - loss: 9.1254
Epoch 11/50
96/96 [==============================] - 2s 21ms/step - loss: 8.9699
Epoch 12/50
96/96 [==============================] - 2s 21ms/step - loss: 8.8124
Epoch 13/50
96/96 [=======================